# 随机森林和AdaBoost

#1 弱分类器和强分类器
最好的分类器是支持向量机,通常正确率为50%左右的是比较若的分类器，而正确率在80%或者90%以上的就是比较好的分类器。我们可以联合使用多个弱分类器，来形成一个强的分类器

#2 分类器的组合算法
- 装袋(bagging)
- 提升(boosting) - Adaboost
- 随机森林

我们可以用多个弱分类器，然后通过这三种不同的框架来实现一个强的分类器

#3 装袋算法和随机森林

##3.1 自助式抽样
我们需要构造多个弱分类器，但是又不能多次重复使用统一分数据(因为这样做只会得到同一个结果)，所以就需要构造不同的数据。用来构造不同的数据集的方法，就是自助式抽样

从总体m条记录中，有放回的抽出m个样本，组成的就是自助式抽样，由于是有放回的，所以抽出的m条记录是可能有重复记录的，所以两个m是不一样的。再一次抽样过程中，某一个样本被抽中的概率是$\frac{1}{m}$，那么它不被抽中的概率就是$1-\frac{1}{m}$，那么在一轮自助式抽样过程中，这个样本都没有被抽中的概率是$p=(1-\frac{1}{m})^m$，那么我们极限$\lim\limits_{m\to\infty}p=\lim\limits_{m\to\infty}(1-\frac{1}{m})^m=e^{-1}\approx0.368$，所以用自助式抽样所抽出的样本，能够覆盖总体样本的$1-0.368=0.632=63.2$%

##3.2 装袋算法
Bagging(Bootstrap aggregating的缩写)算法是最早的集成学习算法，具体步骤可以描述为：
- 利用Bootstrap方法重采样(自助式抽样)，随机产生T个训练集$S_1,S_2,...,S_T$;
- 利用每个训练集，生成对应的决策树$C_1,C_2,...,C_T$；
- 对于测试数据集样本X，利用每个决策树进行测试，得到对应的类别$C_1(X),C_2(X),...,C_T(X)$；
- 采用投票的方法，将T个决策树中输出最多的类别作为测试集样本X所属的类别

##3.3 随机森林
在自助式抽样的基础上，随机抽取几个属性来构成数据，相当于数据和属性都是随机的

本身来说，决策树是强分类器，只不过在数据随机或者属性随机的情况下，决策树就变成了弱分类器，通过若干个弱分类器的组合，我们就形成了一个可以跟支持向量机相媲美的强分类器

#4 AdaBoost算法
使用装袋算法、随机森林和AdaBoost算法，决策树不需要剪枝的
AdaBoost是一种迭代算法，把弱分类器集合成一个更强的分类器

根据每次训练集中每个样本的分类是否正确，以及上次的总体分类的准确率，来确定每个样本的权重(即改变数据分布)。将修改果权值的新数据集送给下层分类器进行训练，最后将每次得到的分类器融合在一起，作为最后的计策分类器
所以，随机森林和装袋算法是可以并行的，但是AdaBoost算法是串行的

##4.1 AdaBoost思想
- 先通过对N个训练样本的学习得到第一个弱分类器；
- 根据本次分类结果制造一个新的N个的训练样本，通过对这个样本的学习得到第二个弱分类器；
- 根据1，2的分类结果制造一个新的N个的训练样本，通过对这个样本的学习得到第三个弱分类器；
- 最终经过提升的强分类器。即某个数据被分到哪一类都要通过多数表决

##4.2 如何制造新的训练样本
用样本权重模拟样本分布：对于分类错误的样本，加大其对应的权重；而对于分类正确的样本，降低其权重，这样分错的样本就会被突显出来，得到一个新的样本分布

##4.3 如何联合多个弱分类器
使用加权的投票机制代替贫困投票机制。让分类效果好的弱分类器具有较大的权重，而分类效果差的分类器具有较小的权重

##4.4 算法
```
循环迭代，直到累计错误率为0{
	更新样本分布D
	获得当前分布下的最好的弱分类器
	计算最好弱分类器的误差率
	计算最好弱分类器的话语权
}
```
误差率用$\varepsilon$表示，话语权用$\alpha$

给定样本$x_1,x_2,...,x_m$，以及对应的类别$y_1,y_2,...,y_m$,其中$x_i \in X$, $y_i \in Y = \{-1,+1\}$,初始化$D_1(i)=\frac{1}{m}$. 从t=1到T：
- 用当前分布训练一个弱分类器$D_t$；
- 计算弱分类器错误率:$\varepsilon_t$,好的分类器误差率应该小余0.5
- 计算话语权$\alpha_t=\frac{1}{2}ln(\frac{1-\varepsilon_t}{\varepsilon_t})$
- 更新分布:$D_{t+1}(i)=\frac{D_t(i)}{Z_t} * 
\begin{cases}
e^{-\alpha_t} & h_t(x_i) = y_i \\
e^{\alpha_t} & h_t(x_i) \neq y_i
\end{cases}=\frac{D_t(i)e^{\alpha_ty_ih_t(x_i)}}{Z_t}$

最后有：$H(x)=sign(\sum_{t=1}^T\alpha_th_t(x))$